In [ ]:
import mode_analyze
reload(mode_analyze)
%pylab inline

In [ ]:
#Setting up models
model=mode_analyze.n3_poly()
ma_3=mode_analyze.ModeAnalyzer(model,'poly_3/poly3.n', n_min=-18, n_max=10, ls=[2,3])

model=mode_analyze.n32_poly()
ma_32=mode_analyze.ModeAnalyzer(model,'poly_32/poly32.n', n_min=0, n_max=10, ls=[2,3])


MM='1'
loc='/home/aleksey/Dropbox/'
base='{0}Msun_tau1_mesa/'.format(MM)
prof=mode_analyze.prep_mesa(base)

ma_3b=mode_analyze.ModeAnalyzer(prof, '/home/aleksey/Dropbox/gyre_work/1Msun_tau1/{0}Msun.n'.format(MM), -105, 10, [2])



In [ ]:
from astropy.table import Table

ns=range(-18, 11)
nsb=range(-105, 11)
ns=np.array(ns)
nsb=np.array(nsb)

ls=[2]
#mode_summary_n3=np.empty([len(ns)*len(ls), 5])
#mode_summary_n3b=np.empty([len(nsb)*len(ls), 5])

fig,ax=plt.subplots(figsize=(10,9))
ax.set_ylim(-5, 0.)
ax.set_xlim(0.1, 20)
ax.set_xlabel(r'$\omega$')
ax.set_ylabel('log(Q)')


tags=[str(nn)+'_2' for nn in ns]
omegas=np.array([ma_3.modes_dict[tt]['omega'] for tt in tags])
Qs=np.array([ma_3.modes_dict[tt]['Q'] for tt in tags])
semilogx(omegas, log10(Qs), 'ks-')

 
##Qs from Lee&Ostriker1986    
compare=[1.894e-3, 2.7e-3, 4.121e-3, 6.263e-3, 9.442e-3, 1.492e-2, 2.470e-2, 
         4.457e-2, 9.955e-2, 0.2372, 0.1226, 0.06163, 0.03498, 0.0211, 0.01322]
semilogx(omegas[(ns>=-9) & (ns<=5)], log10(compare), 'rs', label='lee+ostriker1986')
# ax.legend()



In [ ]:
##Comparing to tidal coupling constants from 

%pylab inline
fig,ax=plt.subplots(figsize=(10,9))
ax.set_xlabel(r'$\eta$')
ax.set_ylabel(r'$T(\eta)$')


dat=np.genfromtxt('coupling_n3.csv',delimiter=',')
order=np.argsort(dat[:,0])

loglog(ma_3.etas, ma_3.tidal_coupling(2), label='Lee&Ostriker1986')
loglog(dat[:,0][order], dat[:,1][order], 'r--', label='Mode analyze')

ax.legend()

In [ ]:
%pylab inline
fig,ax=plt.subplots(figsize=(10,9))
ax.set_xlabel(r'$\eta$')
ax.set_ylabel(r'$T(\eta)$')

dat=np.genfromtxt('coupling_n32.csv',delimiter=',')
order=np.argsort(dat[:,0])

loglog(ma_32.etas, ma_32.tidal_coupling(2), label='Lee&Ostriker1986')
loglog(dat[:,0][order], dat[:,1][order], 'r--', label='Mode analyze')

ax.legend()